In [1]:
#specify GPU to use
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
from sklearn.model_selection import train_test_split
import openml
import category_encoders as ce
import numpy as np
import sklearn

# Load healthcare_insurance_expenses dataset
dataset = openml.datasets.get_dataset(46931, download_data=True, download_qualities=True, download_features_meta_data=True)
X, y, categorical_indicator, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
categorical_feature_indices = [idx for idx, idx_bool in enumerate(categorical_indicator) if idx_bool]

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

print("Training set size:", len(X_train))
print("Validation set size:", len(X_valid))
print("Test set size:", len(X_test))

X_train.head()

Training set size: 856
Validation set size: 214
Test set size: 268


,age,sex,bmi,children,smoker,region
94,33,male,35.750,1,yes,southeast
814,46,male,19.855,0,no,northwest
246,28,female,27.500,2,no,southwest
794,36,female,25.900,1,no,southwest
1239,42,female,36.195,1,no,northwest


In [3]:
categorical_feature_indices

[1, 4, 5]

In [ ]:
from GRANDE import GRANDE

params = {
    'depth': 5,
    'n_estimators': 1024,
    
    'learning_rate_weights': 0.001,
    'learning_rate_index': 0.01,
    'learning_rate_values': 0.05,
    'learning_rate_leaf': 0.15,
    'learning_rate_embedding': 0.01,

    'use_category_embeddings': True,
    'embedding_dim_cat': 8,
    'use_numeric_embeddings': False,
    'embedding_dim_num': 8,
    'embedding_threshold': 1,
    'loo_cardinality': 10,


    'dropout': 0.2,
    'selected_variables': 0.8,
    'data_subset_fraction': 1.0,
    'bootstrap': False,
    'missing_values': False,

    'optimizer': 'adam', #nadam, radam, adamw, adam 
    'cosine_decay_restarts': False,
    'reduce_on_plateau_scheduler': True,
    'label_smoothing': 0.0,
    'use_class_weights': False,
    'focal_loss': False,
    'swa': False,
    'es_metric': True,  # if True use AUC for binary, MSE for regression, val_loss for multiclass


    'epochs': 250,
    'batch_size': 256,
    'early_stopping_epochs': 50,

    'use_freq_enc': False,
    'use_robust_scale_smoothing': False,
    'problem_type': 'regression',
    
    'random_seed': 42,
    'verbose': 2,
}

model_grande = GRANDE(params=params)

model_grande.fit(X=X_train,
          y=y_train,
          X_val=X_valid,
          y_val=y_valid)

preds_grande = model_grande.predict_proba(X_test)

self.params {'depth': 5, 'n_estimators': 1024, 'learning_rate_weights': 0.001, 'learning_rate_index': 0.01, 'learning_rate_values': 0.05, 'learning_rate_leaf': 0.15, 'learning_rate_embedding': 0.01, 'use_category_embeddings': True, 'embedding_dim_cat': 8, 'use_numeric_embeddings': False, 'embedding_dim_num': 8, 'embedding_threshold': 1, 'loo_cardinality': 10, 'dropout': 0.2, 'selected_variables': 0.8, 'data_subset_fraction': 1.0, 'bootstrap': False, 'missing_values': False, 'optimizer': 'adam', 'cosine_decay_restarts': False, 'reduce_on_plateau_scheduler': True, 'label_smoothing': 0.0, 'use_class_weights': False, 'focal_loss': False, 'swa': False, 'es_metric': True, 'epochs': 250, 'batch_size': 256, 'early_stopping_epochs': 50, 'use_freq_enc': False, 'use_robust_scale_smoothing': False, 'problem_type': 'regression', 'random_seed': 42, 'verbose': 2, 'device': 'cuda:0', 'objective': 'regression'}
X_train shape before embeddings (856, 6)
categories [5]
number_of_variables 1 6
number_of_va

/home/smarton/anaconda3/envs/ReMeDe/lib/python3.12/site-packages/torch/_inductor/lowering.py:7242: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(
/home/smarton/anaconda3/envs/ReMeDe/lib/python3.12/site-packages/torch/_inductor/lowering.py:7242: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(


Epoch 001 | TrainLoss: 0.9442 | ValLoss: 0.6983 | MSE: 110805304.0000 | MAE: 7907.7832 | R2: 0.2430 | Time: 6.61s
Epoch 002 | TrainLoss: 0.6745 | ValLoss: 0.4770 | MSE: 75694136.0000 | MAE: 6047.0703 | R2: 0.4829 | Time: 0.02s
Epoch 003 | TrainLoss: 0.4664 | ValLoss: 0.3108 | MSE: 49315516.0000 | MAE: 4471.3096 | R2: 0.6631 | Time: 0.02s
Epoch 004 | TrainLoss: 0.3181 | ValLoss: 0.2167 | MSE: 34391280.0000 | MAE: 4079.7856 | R2: 0.7651 | Time: 0.02s
Epoch 005 | TrainLoss: 0.2304 | ValLoss: 0.1863 | MSE: 29557700.0000 | MAE: 3613.9580 | R2: 0.7981 | Time: 0.02s
Epoch 006 | TrainLoss: 0.2101 | ValLoss: 0.1661 | MSE: 26352892.0000 | MAE: 3145.3518 | R2: 0.8200 | Time: 0.02s
Epoch 007 | TrainLoss: 0.1697 | ValLoss: 0.1398 | MSE: 22176866.0000 | MAE: 2775.7600 | R2: 0.8485 | Time: 0.02s
Epoch 008 | TrainLoss: 0.1484 | ValLoss: 0.1316 | MSE: 20874780.0000 | MAE: 2580.6177 | R2: 0.8574 | Time: 0.02s
Epoch 009 | TrainLoss: 0.1400 | ValLoss: 0.1328 | MSE: 21068986.0000 | MAE: 2652.9009 | R2: 0.8

/home/smarton/anaconda3/envs/ReMeDe/lib/python3.12/site-packages/torch/_inductor/lowering.py:7242: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(
/home/smarton/anaconda3/envs/ReMeDe/lib/python3.12/site-packages/torch/_inductor/lowering.py:7242: UserWarning: 
Online softmax is disabled on the fly since Inductor decides to
split the reduction. Cut an issue to PyTorch if this is an
important use case and you want to speed it up with online
softmax.

  warnings.warn(


In [5]:
def calculate_sample_weights(y_data):
    class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight = 'balanced', classes = np.unique(y_data), y = y_data)
    sample_weights = sklearn.utils.class_weight.compute_sample_weight(class_weight = 'balanced', y =y_data)
    return sample_weights

In [6]:
try:
    y_train = y_train.values.codes.astype(np.float64)
    y_valid = y_valid.values.codes.astype(np.float64)
    y_test = y_test.values.codes.astype(np.float64)
except:
    y_train = y_train.values.astype(np.float64)
    y_valid = y_valid.values.astype(np.float64)
    y_test = y_test.values.astype(np.float64)

In [7]:
binary_indices = []
low_cardinality_indices = []
high_cardinality_indices = []
num_columns = []
for column_index, column in enumerate(X_train.columns):
    if column_index in categorical_feature_indices:
        if len(X_train.iloc[:,column_index].unique()) <= 2:
            binary_indices.append(column)
        if len(X_train.iloc[:,column_index].unique()) < 5:
            low_cardinality_indices.append(column)
        else:
            high_cardinality_indices.append(column)
    else:
        num_columns.append(column)     
cat_columns = [col for col in X_train.columns if col not in num_columns]

In [8]:
if len(num_columns) > 0:
    mean_train_num = X_train[num_columns].mean(axis=0).iloc[0]
    X_train[num_columns] = X_train[num_columns].fillna(mean_train_num)
    X_valid[num_columns] = X_valid[num_columns].fillna(mean_train_num)
    X_test[num_columns] = X_test[num_columns].fillna(mean_train_num)
if len(cat_columns) > 0:
    mode_train_cat = X_train[cat_columns].mode(axis=0).iloc[0]
    X_train[cat_columns] = X_train[cat_columns].fillna(mode_train_cat)
    X_valid[cat_columns] = X_valid[cat_columns].fillna(mode_train_cat)
    X_test[cat_columns] = X_test[cat_columns].fillna(mode_train_cat)

X_train_raw = X_train.copy()
X_valid_raw = X_valid.copy()
X_test_raw = X_test.copy()

In [9]:
encoder_ordinal = ce.OrdinalEncoder(cols=binary_indices)
encoder_ordinal.fit(X_train)
X_train = encoder_ordinal.transform(X_train)
X_valid = encoder_ordinal.transform(X_valid)  
X_test = encoder_ordinal.transform(X_test)       

encoder = ce.LeaveOneOutEncoder(cols=high_cardinality_indices)
encoder.fit(X_train, y_train)
X_train = encoder.transform(X_train)
X_valid = encoder.transform(X_valid)
X_test = encoder.transform(X_test)

encoder = ce.OneHotEncoder(cols=low_cardinality_indices)
encoder.fit(X_train)
X_train = encoder.transform(X_train)
X_valid = encoder.transform(X_valid)
X_test = encoder.transform(X_test)

X_train = X_train.astype(np.float32)
X_valid = X_valid.astype(np.float32)
X_test = X_test.astype(np.float32)

In [10]:
if params['problem_type'] == 'regression':
    from xgboost import XGBRegressor
    model_xgb = XGBRegressor(n_estimators=1000, early_stopping_rounds=20)
    model_xgb.fit(X_train, 
                    y_train, 
                    eval_set=[(X_valid, y_valid)], 
                )
    preds_xgb = model_xgb.predict(X_test)
else:
    from xgboost import XGBClassifier
    model_xgb = XGBClassifier(n_estimators=1000, early_stopping_rounds=20)
    model_xgb.fit(X_train, 
                    y_train, 
                    #sample_weight=calculate_sample_weights(y_train), 
                    eval_set=[(X_valid, y_valid)], 
                    #sample_weight_eval_set=[calculate_sample_weights(y_valid)]
                )


    preds_xgb = model_xgb.predict_proba(X_test)

[0]	validation_0-rmse:9117.37519
[1]	validation_0-rmse:7182.91726
[2]	validation_0-rmse:6008.67928
[3]	validation_0-rmse:5384.74130
[4]	validation_0-rmse:5041.28295
[5]	validation_0-rmse:4821.53449
[6]	validation_0-rmse:4714.24431
[7]	validation_0-rmse:4660.87726
[8]	validation_0-rmse:4676.10315
[9]	validation_0-rmse:4700.87577
[10]	validation_0-rmse:4677.24837
[11]	validation_0-rmse:4667.92836
[12]	validation_0-rmse:4681.76173
[13]	validation_0-rmse:4687.39543
[14]	validation_0-rmse:4754.37928
[15]	validation_0-rmse:4738.09888
[16]	validation_0-rmse:4735.64695
[17]	validation_0-rmse:4751.13172
[18]	validation_0-rmse:4773.50335
[19]	validation_0-rmse:4790.19284
[20]	validation_0-rmse:4773.47564
[21]	validation_0-rmse:4774.69551
[22]	validation_0-rmse:4778.65167
[23]	validation_0-rmse:4775.53956
[24]	validation_0-rmse:4781.68505
[25]	validation_0-rmse:4795.20388
[26]	validation_0-rmse:4829.21788
[27]	validation_0-rmse:4837.16893


In [11]:
if params['problem_type'] == 'regression':
    from catboost import CatBoostRegressor, Pool

    model_catboost = CatBoostRegressor(n_estimators=1000, 
                                        early_stopping_rounds=20)
    train_data = Pool(
            data=X_train_raw,
            label=y_train,
            cat_features=categorical_feature_indices,
        )

    eval_data = Pool(
            data=X_valid_raw,
            label=y_valid,
            cat_features=categorical_feature_indices,
        )

    model_catboost.fit(X=train_data, 
                    eval_set=eval_data)




    preds_catboost = model_catboost.predict(X_test_raw)
else:
    from catboost import CatBoostClassifier, Pool

    model_catboost = CatBoostClassifier(n_estimators=1000, 
                                        early_stopping_rounds=20)
    train_data = Pool(
            data=X_train_raw,
            label=y_train,
            cat_features=categorical_feature_indices,
            #weight=calculate_sample_weights(y_train)
        )

    eval_data = Pool(
            data=X_valid_raw,
            label=y_valid,
            cat_features=categorical_feature_indices,
            #weight=calculate_sample_weights(y_valid),
        )

    model_catboost.fit(X=train_data, 
                    eval_set=eval_data)



    preds_catboost = model_catboost.predict_proba(X_test_raw)


Learning rate set to 0.049693
0:	learn: 12135.8956932	test: 11637.9908345	best: 11637.9908345 (0)	total: 48.8ms	remaining: 48.8s
1:	learn: 11663.8838516	test: 11175.5245901	best: 11175.5245901 (1)	total: 50.8ms	remaining: 25.4s
2:	learn: 11273.2960222	test: 10835.8030762	best: 10835.8030762 (2)	total: 52.5ms	remaining: 17.5s
3:	learn: 10889.3829166	test: 10474.9396901	best: 10474.9396901 (3)	total: 53.4ms	remaining: 13.3s
4:	learn: 10532.8002095	test: 10132.1552031	best: 10132.1552031 (4)	total: 55ms	remaining: 10.9s
5:	learn: 10196.7503986	test: 9816.3979358	best: 9816.3979358 (5)	total: 56.6ms	remaining: 9.38s
6:	learn: 9836.9135106	test: 9465.9962973	best: 9465.9962973 (6)	total: 58.7ms	remaining: 8.32s
7:	learn: 9523.0102491	test: 9176.2224294	best: 9176.2224294 (7)	total: 59.7ms	remaining: 7.4s
8:	learn: 9234.9423371	test: 8905.7040385	best: 8905.7040385 (8)	total: 60.2ms	remaining: 6.63s
9:	learn: 8946.3797800	test: 8636.1173694	best: 8636.1173694 (9)	total: 61ms	remaining: 6.04s

In [12]:
if params['problem_type'] == 'binary':
    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_grande[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_grande[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_grande[:,1], average='macro', multi_class='ovo')

    print('Accuracy GRANDE:', accuracy)
    print('F1 Score GRANDE:', f1_score)
    print('ROC AUC GRANDE:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_xgb[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_xgb[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_xgb[:,1], average='macro', multi_class='ovo')

    print('Accuracy XGB:', accuracy)
    print('F1 Score XGB:', f1_score)
    print('ROC AUC XGB:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_catboost[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_catboost[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_catboost[:,1], average='macro', multi_class='ovo')

    print('Accuracy CatBoost:', accuracy)
    print('F1 Score CatBoost:', f1_score)
    print('ROC AUC CatBoost:', roc_auc)
    print('\n')
elif params['problem_type'] == 'multiclass':
    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_grande, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_grande, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_grande, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy GRANDE:', accuracy)
    print('F1 Score GRANDE:', f1_score)
    print('ROC AUC GRANDE:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_xgb, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_xgb, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_xgb, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy XGB:', accuracy)
    print('F1 Score XGB:', f1_score)
    print('ROC AUC XGB:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_catboost, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_catboost, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_catboost, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy CatBoost:', accuracy)
    print('F1 Score CatBoost:', f1_score)
    print('ROC AUC CatBoost:', roc_auc)
    print('\n')
else:
    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, preds_grande)
    root_mean_squared_error = np.sqrt(((y_test - preds_grande) ** 2).mean())
    r2_score = sklearn.metrics.r2_score(y_test, preds_grande)

    print('MAE GRANDE:', mean_absolute_error)
    print('RMSE GRANDE:', root_mean_squared_error)
    print('R2 Score GRANDE:', r2_score)
    print('\n')

    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, preds_xgb)
    root_mean_squared_error = np.sqrt(((y_test - preds_xgb) ** 2).mean())
    r2_score = sklearn.metrics.r2_score(y_test, preds_xgb)

    print('MAE XGB:', mean_absolute_error)
    print('RMSE XGB:', root_mean_squared_error)
    print('R2 Score XGB:', r2_score)
    print('\n')

    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, preds_catboost)
    root_mean_squared_error = np.sqrt(((y_test - preds_catboost) ** 2).mean())
    r2_score = sklearn.metrics.r2_score(y_test, preds_catboost)

    print('MAE CatBoost:', mean_absolute_error)
    print('RMSE CatBoost:', root_mean_squared_error)
    print('R2 Score CatBoost:', r2_score)
    print('\n')

MAE GRANDE: 2481.6191651649956
RMSE GRANDE: 3761.773606283763
R2 Score GRANDE: 0.8653776732830805


MAE XGB: 2437.6996424431554
RMSE XGB: 3930.3302577846944
R2 Score XGB: 0.8530431372729947


MAE CatBoost: 2194.5464048630265
RMSE CatBoost: 3637.838113587122
R2 Score CatBoost: 0.8741020902072107


